<a href="https://colab.research.google.com/github/alanntl/SELGO-LITE/blob/alanntl/SLEGO_LITE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SLEGO Project: UNSW CSE PhD Research - Alan Siu Lung Ng

# Connect this Notebook to your GoogleDrive

In [2]:
%%time
from google.colab import drive
from google.colab import files

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive
CPU times: user 223 ms, sys: 40.9 ms, total: 264 ms
Wall time: 4.61 s


# Clone SLEGO repo

In [4]:
%%time
import os
import sys
from IPython.display import clear_output
import subprocess

# Path to your repository
repo_path = '/content/drive/MyDrive/SLEGO'
repo_url = 'https://github.com/alanntl/SELGO-LITE.git'

# Function to run subprocess commands with error handling
def run_command(command, check=True, **kwargs):
    try:
        subprocess.run(command, check=check, **kwargs)
    except subprocess.CalledProcessError as e:
        print(f"Error running command {' '.join(command)}: {e}")
        return None
    return True

# Ensure the repository path exists
if not os.path.exists(repo_path):
    if run_command(['git', 'clone', repo_url, repo_path]):
        print("Repository cloned.")
else:
    # Change to the repository directory
    os.chdir(repo_path)

    # Fetch latest changes from the repository
    if run_command(['git', 'fetch']):
        # Check for new or changed directories
        result = subprocess.run(['git', 'diff', '--name-only', 'HEAD', 'origin/master'], capture_output=True, text=True)
        changed_files = result.stdout.splitlines()
        directory_changes = any(os.path.dirname(f) for f in changed_files)

        if directory_changes:
            # If there are directory changes, pull the updates
            if run_command(['git', 'pull']):
                print("Repository updated with new directory changes.")
        else:
            print("No directory changes detected; no update necessary.")


Error running command git fetch: Command '['git', 'fetch']' returned non-zero exit status 1.
CPU times: user 5.86 ms, sys: 418 µs, total: 6.27 ms
Wall time: 805 ms


# Setup virtural library

In [ ]:

!pip install virtualenv

# Assuming you want to use 'slego_env' as your virtual environment
!virtualenv "/content/drive/MyDrive/slego_env_v0_0_1"

slego_env = "/content/drive/MyDrive/slego_env_v0_0_1"
requirements_file = "/content/drive/MyDrive/SLEGO/requirements.txt"

# Activate the virtual environment and install requirements
!source "$slego_env/bin/activate"; pip install -r "$requirements_file"

# Append the path to sys.path
import sys
sys.path.append(f"{slego_env}/lib/python3.10/site-packages")


created virtual environment CPython3.10.12.final.0-64 in 6380ms
  creator CPython3Posix(dest=/content/drive/MyDrive/slego_env_v0_0_1, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.0, setuptools==69.5.1, wheel==0.43.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Setup the app foldersystem for slego

In [5]:
# This will prompt for authorization and mount your Google Drive.
drive.mount('/content/drive', force_remount=True)

folder_path = '/content/drive/MyDrive/SLEGO/slegospace'

# Now you can reference subfolders and files relative to the top-level folder
dataspace = '/dataspace/'  # This is equivalent to '/content/drive/MyDrive/SLEGO/slegospace/dataspace'
recordspace = '/recordspace/'  # Equivalent to '/content/drive/MyDrive/SLEGO/slegospace/recordspace'
functionspace = '/functionspace/'  # Equivalent to '/content/drive/MyDrive/SLEGO/slegospace/functionspace'
knowledgespace = '/knowledgespace/'  # Equivalent to '/content/drive/MyDrive/SLEGO/slegospace/knowledgespace'

files.view(folder_path)
os.chdir(folder_path)

Mounted at /content/drive


<IPython.core.display.Javascript object>

# Install Libraries on Colab temporarily or install them into google drive

In [7]:
%%time
# uncomment below codes if you dont want to save these dependencies on google drive
# !pip install -r /content/drive/MyDrive/SLEGO/requirements.txt

# uncomment below codes if you want to save these dependencies on google drive and reuse it next time, Path to the directory on Google Drive where the dependencies are stored
dependencies_dir = "/content/drive/MyDrive/SLEGO_dependencies_v0_0_2"
requirements_file = "/content/drive/MyDrive/SLEGO/requirements.txt"


# # Uncomment below if you wanna save the python dependencies on your google drive, once you have saved it, you can comment this again to avoid reinstallation

if not os.path.exists(dependencies_dir):
    # Install dependencies if the directory does not exist
    print("Installing dependencies...")
    !pip install --target="$dependencies_dir" -r $requirements_file
else:
    # If the directory exists, you might want to update or ensure all packages are installed
    pass
    # print("Updating/installing packages...")
    # !pip install --target="$dependencies_dir" -r $requirements_file

# Append the directory to sys.path
if dependencies_dir not in sys.path:
    sys.path.append(dependencies_dir)

print("Setup complete. Packages are ready to use.")

clear_output()


CPU times: user 3.56 ms, sys: 887 µs, total: 4.45 ms
Wall time: 4.06 ms


# Import Libraries
Import all the related libraries

In [8]:
%%time
import panel as pn
import inspect
import ast  # For safely evaluating the input string
import re
import importlib
import json
import io
import time
import param
import json
from datetime import datetime
import itertools

# if colab or vscode or jupyter
if 'google.colab' in str(get_ipython()):
    pn.extension(comms='colab')
elif 'vscode' in str(get_ipython()):
    pn.extension(comms='vscode')
elif 'jupyter' in str(get_ipython()):
    pn.extension(comms='jupyter')

pn.extension(sizing_mode = 'stretch_both')

CPU times: user 1.8 s, sys: 64.3 ms, total: 1.86 s
Wall time: 27.8 s


# Select which modules to import

In [9]:
# see load time
%%time
def delete_func_file(func_file_path):
    # Check if the file exists
    if os.path.exists(func_file_path):
        # Delete the file
        os.remove(func_file_path)
        #print(f"File {func_file_path} has been deleted.")
    else:
        print(f"No file named {func_file_path} found.")


# Step 3: Get the list of .py files in the folder
py_files = [file for file in os.listdir(folder_path+functionspace) if file.endswith('.py')]

# New Step: Check if func.py exists and delete it
func_file_path = 'func.py'
delete_func_file(func_file_path)

funcfilecombo = pn.widgets.MultiChoice(name='Select Function',
                                       value=['util.py',
                                              'func_data_preprocss.py',
                                              'func_moving_avg_plot.py',
                                              'llm.py',
                                              'func_eda.py',
                                              'func_uci_dataset.py',
                                              'webscrape.py',
                                              'func_arxiv.py',
                                              'func_backtest.py',
                                              'func_autogluon.py'],
                                       options=py_files, height=200)

# create funcfilecombo_change function
def funcfilecombo_change(event):
    delete_func_file(func_file_path)

    py_files = funcfilecombo.value
    # Step 4 (Modified): Create a new file named func.py in the current repository
    with open(func_file_path, 'w') as func_file:
        # Step 5: Iterate over each .py file and append its content to func.py
        for py_file in py_files:
            file_path = os.path.join(folder_path+functionspace, py_file)
            with open(file_path, 'r') as file:
                func_file.write(file.read() + '\n')
    import func
    importlib.reload(func)


# param watch
funcfilecombo.param.watch(funcfilecombo_change, 'value')

display(funcfilecombo)

# call the event
funcfilecombo_change(None)
!ls

MultiChoice(height=200, name='Select Function', options=['util.py', 'func_pycaret....], sizing_mode='stretch_width', value=['util.py', 'func_data_pre...])

No file named func.py found.
Imported v0.1.802. After importing autoviz, you must run '%matplotlib inline' to display charts inline.
    AV = AutoViz_Class()
    dfte = AV.AutoViz(filename, sep=',', depVar='', dfte=None, header=0, verbose=1, lowess=False,
               chart_format='svg',max_rows_analyzed=150000,max_cols_analyzed=30, save_plot_dir=None)
AutogluonModels  dataspace  func.py  functionspace  knowledgespace  __pycache__  recordspace
CPU times: user 7.11 s, sys: 2.31 s, total: 9.42 s
Wall time: 7min 44s


# SLEGO APP

In [10]:
%%time
import func
importlib.reload(func)

funccombo = func._create_multi_select_combobox(func)
input_text = pn.widgets.TextAreaInput(value='', placeholder='input the parameters', height=300)
compute_btn = pn.widgets.Button(name='Compute', height =50,  button_type='primary')
output_text= pn.widgets.TextAreaInput(value='',placeholder= 'Results will be shown here')
savepipe_btn = pn.widgets.Button(name='Save Pipeline', height =50)
pipeline_text= pn.widgets.TextInput(value='', placeholder='Input Pipeline Name', height=50)

pipeline_dict = {}


def get_doc_string(pipeline):
    text = input_text.value
    output=''
    data = ast.literal_eval(text)
    data.keys()
    # loop keys
    for key in data.keys():
        output +='#######'+str(key)+'#######\n'
        try:
            output += eval(f'func.{key}.__doc__')+'\n'
        except:
            output += 'No docstring found for this function\n'
    return output

def input_text_change(event):
    global pipeline_dict
    text = re.sub(r'\bfalse\b', 'False', input_text.value, flags=re.IGNORECASE)
    try:
        pipeline_dict = ast.literal_eval(text)
        #print(pipeline_dict)
    except ValueError as e:
        output_text.value = f'Error parsing input: {e}'


def funccombo_change(event):
    output_text.value = ''
    list_funcs = funccombo.value
    list_params =[]

    for funcchoice in funccombo.value:
      function=  eval('func.'+funcchoice)
      list_params.append(func._extract_parameter(function))

    funcs_params = dict(zip(list_funcs,list_params))
    formatted_data = json.dumps(funcs_params, indent=5)
    input_text.value = str(formatted_data)
    output_text.value = get_doc_string(input_text.value)


def save_record(space, data, pipeline_name=None):
    """
    Saves the given data structure as a JSON file named with the current datetime.

    Parameters:
    - data: The data structure to be saved as JSON.
    """
    if pipeline_name is None:
      # Generate a filename based on the current datetime, e.g., "record_20230405_153000.json"
      filename = datetime.now().strftime("record_%Y%m%d_%H%M%S.json")
    else:
      filename = pipeline_name+'.json'

    # Define the full path where you want to save the file, here assuming current directory
    full_path = './'+space+'/' + filename

    # Serialize and save the data structure to a file
    with open(full_path, "w") as file:
        json.dump(data, file, indent=5)


def compute_btn_clicked(event):
    input_text_change(None)
    time.sleep(1)
    output_text.value = ''

    for function_name, parameters in pipeline_dict.items():
        module = 'func.'
        start_time = time.time()  # Start the timer

        # Dynamically execute the function
        function = eval(module + function_name)
        result = function(**parameters)
        # Convert result to string
        result_string = str(result)
        # Create an iterator for words
        words_iterator = iter(result_string.split())
        # Use itertools.islice to get the first 1000 words without creating a full list
        first_x_words = itertools.islice(words_iterator, 500)
        # Join the words and append to output_text.value
        # Computation time
        compute_time = time.time() - start_time

        output_text.value += f"\n===================={function_name}====================\n\n"
        output_text.value += f"Function computation Time: {compute_time:.4f} seconds\n"
        output_text.value += f"\n"
        output_text.value += " ".join(first_x_words)

    save_record('recordspace', pipeline_dict)

def save_pipeline(event):
  if pipeline_text.value == '':
    pipeline_name = '__'
  else:
    pipeline_name = pipeline_text.value
  text = input_text.value
  text = re.sub(r'\bfalse\b', 'False', input_text.value, flags=re.IGNORECASE)
  data = ast.literal_eval(text)
  save_record('knowledgespace', data, pipeline_name)

# save output into specific folder
funccombo.param.watch(funccombo_change, 'value')
input_text.param.watch(input_text_change, 'value')

compute_btn.on_click(compute_btn_clicked)
savepipe_btn.on_click(save_pipeline)
app = pn.Row(pn.Column(funccombo,compute_btn,pn.Row(savepipe_btn,pipeline_text), input_text), pn.Column(output_text))

CPU times: user 10.3 ms, sys: 159 µs, total: 10.4 ms
Wall time: 12.4 ms


# Display the App

In [11]:
%%time
# import warnings
# warnings.filterwarnings('ignore', category=UserWarning, message='.*findfont.*')
# Creating a Panel column to display the combobox and the text area
# sk-ThBFYGul5uQJN13tHYgkT3BlbkFJNrFzdHY368549fywvav
app

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.58 µs


Row(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_both')
        [0] MultiChoice(height=150, name='Functions:', options=['autogluon_model_predict'...], sizing_mode='stretch_width')
        [1] Button(button_type='primary', height=50, name='Compute', sizing_mode='stretch_width')
        [2] Row(sizing_mode='stretch_both')
            [0] Button(height=50, name='Save Pipeline', sizing_mode='stretch_width')
            [1] TextInput(height=50, placeholder='Input Pipeline Name', sizing_mode='stretch_width')
        [3] TextAreaInput(height=300, placeholder='input the parameters', sizing_mode='stretch_width')
    [1] Column(sizing_mode='stretch_both')
        [0] TextAreaInput(placeholder='Results will b..., sizing_mode='stretch_both')

In [12]:
import sys
sys.path.append("/content/drive/MyDrive/colab_env/lib/python3.10/site-packages")